# Data Quality Assessment


Sprocket Central Pty Ltd , a medium size bikes & cycling accessories organisation, has approached Tony Smith (Partner) in KPMG’s Lighthouse & Innovation Team. Sprocket Central Pty Ltd  is keen to learn more about KPMG’s expertise in its Analytics, Information & Modelling team. 
Primarily, Sprocket Central Pty Ltd needs help with its customer and transactions data. The organisation has a large dataset relating to its customers, but their team is unsure how to effectively analyse it to help optimise its marketing strategy. 
The client provided KPMG with 3 datasets:
    * Customer Demographic 
    * Customer Addresses
    * Transactions data in the past 3 months
**Target: identifying the data quality issues and how this may impact our analysis going forward**

# Load dataset

In [1]:
!ls

DataQualityAssessment.ipynb            Module_1_Answer.pdf
KPMG_VI_New_raw_data_update_final.xlsx Task1.md


In [8]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#load each tab into dataframe
#skip the first row of each tab
Trans_df = pd.read_excel (r'KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='Transactions',header = 1)
NewCustomerList_df = pd.read_excel (r'KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='NewCustomerList',header = 1)
CusDemographic_df = pd.read_excel (r'KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerDemographic',header = 1)
CusAddress_df = pd.read_excel (r'KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerAddress',header = 1)

In [3]:
#funtions for EDA
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('path/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

---
# Data Cleaning
## Transactions Table

In [37]:
Trans_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [38]:
Trans_df.info()
Trans_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

,transaction_id,product_id,customer_id,online_order,list_price,standard_cost,product_first_sold_date
count,20000.000000,20000.00000,20000.000000,19640.000000,20000.000000,19803.000000,19803.000000
mean,10000.500000,45.36465,1738.246050,0.500458,1107.829449,556.046951,38199.776549
std,5773.647028,30.75359,1011.951046,0.500013,582.825242,405.955660,2875.201110
min,1.000000,0.00000,1.000000,0.000000,12.010000,7.210000,33259.000000
25%,5000.750000,18.00000,857.750000,0.000000,575.270000,215.140000,35667.000000
50%,10000.500000,44.00000,1736.000000,1.000000,1163.890000,507.580000,38216.000000
75%,15000.250000,72.00000,2613.000000,1.000000,1635.300000,795.100000,40672.000000
max,20000.000000,100.00000,5034.000000,1.000000,2091.470000,1759.850000,42710.000000


In [39]:
len(Trans_df.customer_id.unique())

3494

In [40]:
print(Trans_df.transaction_date.max())
print(Trans_df.transaction_date.min())

2017-12-30 00:00:00
2017-01-01 00:00:00


In [41]:
missing_zero_values_table(Trans_df)

Your selected dataframe has 13 columns and 20000 Rows.
There are 7 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
online_order,9811,360,1.8,10171,50.9,float64
brand,0,197,1.0,197,1.0,object
product_line,0,197,1.0,197,1.0,object
product_class,0,197,1.0,197,1.0,object
product_size,0,197,1.0,197,1.0,object
standard_cost,0,197,1.0,197,1.0,float64
product_first_sold_date,0,197,1.0,197,1.0,float64


### Data cleaning
**Fill missing values in `online_order` column based on it's distribution**

In [42]:
online_dis = Trans_df.online_order.value_counts(normalize=True)
online_dis

1.0    0.500458
0.0    0.499542
Name: online_order, dtype: float64

In [43]:
missing_onlineOrder = Trans_df['online_order'].isnull()
Trans_df.loc[missing_onlineOrder,'online_order'] = np.random.choice(online_dis.index, size=len(Trans_df[missing_onlineOrder]),p=online_dis.values)

In [44]:
# drop missing values in other cols
Trans_df.dropna(inplace = True)
Trans_df.shape

(19803, 13)

In [45]:
# check the missing value in Trans_df
missing_zero_values_table(Trans_df)

Your selected dataframe has 13 columns and 19803 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


## NewCustomerList

In [46]:
#drop Unnamed cols
Unnamed_col = [col for col in NewCustomerList_df.columns if 'Unnamed' in col]
NewCustomerList_df.drop(columns=Unnamed_col,inplace = True)

In [47]:
NewCustomerList_df.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


In [48]:
NewCustomerList_df.info()
NewCustomerList_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            894 non-null    object        
 6   job_industry_category                1000 non-null   object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

,past_3_years_bike_related_purchases,tenure,postcode,property_valuation,Rank,Value
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,49.836000,11.388000,3019.227000,7.397000,498.819000,0.881714
std,27.796686,5.037145,848.895767,2.758804,288.810997,0.293525
min,0.000000,0.000000,2000.000000,1.000000,1.000000,0.340000
25%,26.750000,7.000000,2209.000000,6.000000,250.000000,0.649531
50%,51.000000,11.000000,2800.000000,8.000000,500.000000,0.860000
75%,72.000000,15.000000,3845.500000,9.000000,750.250000,1.075000
max,99.000000,22.000000,4879.000000,12.000000,1000.000000,1.718750


In [49]:
missing_zero_values_table(NewCustomerList_df)

Your selected dataframe has 18 columns and 1000 Rows.
There are 3 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
job_title,0,106,10.6,106,10.6,object
last_name,0,29,2.9,29,2.9,object
DOB,0,17,1.7,17,1.7,datetime64[ns]


### Data cleaning

**check the DOB and create new feature `age`**

In [50]:
print(NewCustomerList_df.DOB.max())
print(NewCustomerList_df.DOB.min())

2002-02-27 00:00:00
1938-06-08 00:00:00


In [51]:
now = pd.Timestamp('now')
NewCustomerList_df['age'] = (now - NewCustomerList_df['DOB']).astype('<m8[Y]')

In [52]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(NewCustomerList_df[['age']])
X = imputer.transform(NewCustomerList_df[['age']])

In [53]:
NewCustomerList_df['age'] = X

**Fill missing values based on distribution**

In [54]:
def fillingOnDistribute(df_missing):
    dis = df_missing.value_counts(normalize=True)
    missing = df_missing.isnull()
    df_missing.loc[missing] = np.random.choice(dis.index, size=sum(missing),p=dis.values)
    return df_missing

In [55]:
NewCustomerList_df['job_industry_category'] = fillingOnDistribute(NewCustomerList_df['job_industry_category'])

/Users/henanli/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [56]:
NewCustomerList_df['job_title'] = fillingOnDistribute(NewCustomerList_df['job_title'])

---
## CustomerDemographic

In [58]:
CusDemographic_df.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,66.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,39.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,66.0
3,4,Talbot,NaN,Male,33,1961-10-03,Senior Financial Analyst,IT,Mass Customer,N,No,7.0,58.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,IT,Affluent Customer,N,Yes,8.0,43.0


In [59]:
CusDemographic_df.info()
CusDemographic_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          3999 non-null   int64         
 1   first_name                           3999 non-null   object        
 2   last_name                            3874 non-null   object        
 3   gender                               3999 non-null   object        
 4   past_3_years_bike_related_purchases  3999 non-null   int64         
 5   DOB                                  3912 non-null   datetime64[ns]
 6   job_title                            3999 non-null   object        
 7   job_industry_category                3999 non-null   object        
 8   wealth_segment                       3999 non-null   object        
 9   deceased_indicator                   3999 non-null   object        
 10  owns_car    

,customer_id,past_3_years_bike_related_purchases,tenure,age
count,3999.000000,3999.000000,3999.000000,3999.000000
mean,2000.991748,48.887472,10.654652,42.458365
std,1154.570372,28.718151,5.596988,12.486459
min,1.000000,0.000000,1.000000,18.000000
25%,1001.500000,24.000000,6.000000,33.000000
50%,2001.000000,48.000000,10.654652,42.000000
75%,3000.500000,73.000000,15.000000,52.000000
max,4000.000000,99.000000,22.000000,88.000000


In [60]:
len(CusDemographic_df.customer_id.unique())

3999

In [61]:
print(CusDemographic_df.DOB.max())
print(CusDemographic_df.DOB.min())

2002-03-11 00:00:00
1931-10-23 00:00:00


In [62]:
missing_zero_values_table(CusDemographic_df)

Your selected dataframe has 13 columns and 3999 Rows.
There are 2 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
last_name,0,125,3.1,125,3.1,object
DOB,0,87,2.2,87,2.2,datetime64[ns]


In [63]:
CusDemographic_df.DOB[CusDemographic_df.DOB < datetime(1900,1,1)]

Series([], Name: DOB, dtype: datetime64[ns])

### Data cleaning

**Remove default col and records with invaild DOB**

In [10]:
CusDemographic_df.drop(columns=['default'],inplace = True)

In [15]:
CusDemographic_df.drop(CusDemographic_df.DOB[CusDemographic_df.DOB < datetime(1900,1,1)].index, inplace = True)

In [20]:
CusDemographic_df['job_industry_category'] = fillingOnDistribute(CusDemographic_df['job_industry_category'])

In [21]:
CusDemographic_df['job_title'] = fillingOnDistribute(CusDemographic_df['job_title'])

In [24]:
now = pd.Timestamp('now')
CusDemographic_df['age'] = (now - CusDemographic_df['DOB']).astype('<m8[Y]')

In [ ]:
CusDemographic_df.gender.replace(to_replace=['Femal','F'], value='Female', inplace = True)
CusDemographic_df.gender.replace(to_replace=['M','U'], value='Male', inplace = True)

In [29]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(CusDemographic_df[['age']])
X = imputer.transform(CusDemographic_df[['age']])
CusDemographic_df['age'] = X

In [32]:
imputer = SimpleImputer(strategy = 'mean')
imputer.fit(CusDemographic_df[['tenure']])
X = imputer.transform(CusDemographic_df[['tenure']])
CusDemographic_df['tenure'] = X

In [64]:
missing_zero_values_table(CusDemographic_df)

Your selected dataframe has 13 columns and 3999 Rows.
There are 2 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
last_name,0,125,3.1,125,3.1,object
DOB,0,87,2.2,87,2.2,datetime64[ns]


---
## CustomerAddress

In [21]:
CusAddress_df.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [22]:
CusAddress_df.info()
CusAddress_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


,customer_id,postcode,property_valuation
count,3999.000000,3999.000000,3999.000000
mean,2003.987997,2985.755939,7.514379
std,1154.576912,844.878364,2.824663
min,1.000000,2000.000000,1.000000
25%,1004.500000,2200.000000,6.000000
50%,2004.000000,2768.000000,8.000000
75%,3003.500000,3750.000000,10.000000
max,4003.000000,4883.000000,12.000000


In [23]:
missing_zero_values_table(CusAddress_df)

Your selected dataframe has 6 columns and 3999 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


In [39]:
CusAddress_df.state.unique()

array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object)

In [11]:
len(CusAddress_df.customer_id.unique())

3999

In [35]:
CusAddress_df.replace(to_replace ="Victoria",value ="VIC", inplace = True)
CusAddress_df.replace(to_replace ="New South Wales",value ="NSW", inplace = True)

In [36]:
CusAddress_df.state.unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)

---
## Matching Customer_id

In [68]:
#Customer_id in CusDemographic_df:
Customer_id_Demograph = CusDemographic_df.customer_id.unique().tolist()

In [69]:
#Customer_id in CusAddress_df:
Customer_id_Address = CusAddress_df.customer_id.unique()

In [76]:
#Customer_id in Trans_df:
Trans_df[~Trans_df['customer_id'].isin(Customer_id_Demograph)]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date


In [75]:
CusAddress_df[~CusAddress_df['customer_id'].isin(Customer_id_Demograph)]

,customer_id,address,postcode,state,country,property_valuation


### remove record having `Customer_id ` on in `CusDemographic_df`

In [73]:
Trans_df.drop(Trans_df[~Trans_df['customer_id'].isin(Customer_id_Demograph)].index, inplace = True)

In [74]:
CusAddress_df.drop(CusAddress_df[~CusAddress_df['customer_id'].isin(Customer_id_Demograph)].index, inplace = True)

---
## Save data in database

In [145]:
from sqlalchemy import create_engine
import pymysql
engine = create_engine('', echo=False)

In [147]:
conn = engine.raw_connection()

In [148]:
df = pd.read_sql("SELECT * FROM test;",con = conn) 

In [116]:
Trans_df.to_csv('Transaction.csv',index=False)
CusDemographic_df.to_csv('CusDemographic.csv',index=False)
NewCustomerList_df.to_csv('NewCustomerList.csv',index=False)
CusAddress_df.to_csv('CusAddress.csv',index=False)